### Import Required Libraries and Set Up Environment Variables

In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [5]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = url + "fq=" + filter_query + "&sort=" + sort + "&fl=" + field_list + "&begin_date=" + begin_date + "&end_date=" + end_date + "&api-key=" + nyt_api_key
print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=C0j5jGKD8ztrDtv74OBBAgAGSaqCkFO7


In [7]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page_number in range(20):
    # create query with a page number
    combined_url = url + "&page=" + str(page_number)

    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(combined_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        docs = reviews["response"]["docs"]
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in docs:
            reviews_list.append(review)
            print(review)
    except:
        # Print the page number that had no results then break from the loop
        print("No more results in page: " + str(page_number))
        break
        
        


{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}], 'organization': None}, 'word_count': 295

In [8]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
json_response = json.dumps(reviews_list[:5], indent=4)
print(json_response)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [9]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

In [10]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

In [11]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)

In [12]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].tolist()
print(titles)

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [13]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [14]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0
request_limit = 50

# Loop through the titles
for review_title in titles:
    # Check if we need to sleep before making a request
    if request_counter > request_limit:
        time.sleep(1)
        print("Application is sleeping for 1 second")
        request_counter = 0
    else:
        # Add 1 to the request counter
        request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    full_url_tmbd = url + review_title + tmdb_key_string
    tmdb_title = requests.get(full_url_tmbd).json()


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = tmdb_title["results"][0]["id"]

        # Make a request for a the full movie details
        movie_details_url = "https://api.themoviedb.org/3/movie/" + str(movie_id) + "?api_key=" + tmdb_api_key
        # Execute "GET" request with url
        current_movie = requests.get(movie_details_url).json()

        # Extract the genre names into a list
        current_movie["genres"] = [f"{item['name']}" for item in current_movie["genres"]]

        # Extract the spoken_languages' English name into a list
        current_movie["spoken_languages"] = [f"{item['english_name']}" for item in current_movie["spoken_languages"]]

        # Extract the production_countries' name into a list
        current_movie["production_countries"] = [f"{item['name']}" for item in current_movie["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        dictionary_entry = {}
        dictionary_entry["title"] = current_movie["title"]
        dictionary_entry["original_title"] = current_movie["original_title"]
        dictionary_entry["budget"] = current_movie["budget"]
        dictionary_entry["original_language"] = current_movie["original_language"]
        dictionary_entry["homepage"] = current_movie["homepage"]
        dictionary_entry["overview"] = current_movie["overview"]
        dictionary_entry["popularity"] = current_movie["popularity"]
        dictionary_entry["runtime"] = current_movie["runtime"]
        dictionary_entry["revenue"] = current_movie["revenue"]
        dictionary_entry["release_date"] = current_movie["release_date"]
        dictionary_entry["vote_average"] = current_movie["vote_average"]
        dictionary_entry["vote_count"] = current_movie["vote_count"]
        dictionary_entry["genres"] = current_movie["genres"]
        dictionary_entry["spoken_languages"] = current_movie["spoken_languages"]
        dictionary_entry["production_countries"] = current_movie["production_countries"]


        tmdb_movies_list.append(dictionary_entry)
        # Print out the title that was found
        print("Movie title found, ID: " + str(movie_id) + ", Title: " + dictionary_entry["title"])

    except:
        print("Title not found")



Movie title found, ID: 743040, Title: The Attachment Diaries
Title not found
Movie title found, ID: 887580, Title: You Can Live Forever
Movie title found, ID: 813726, Title: A Tourist's Guide to Love
Movie title found, ID: 803694, Title: Other People's Children
Movie title found, ID: 846961, Title: One True Loves
Movie title found, ID: 965042, Title: The Lost Weekend: A Love Story
Movie title found, ID: 855263, Title: A Thousand and One
Movie title found, ID: 703451, Title: Your Place or Mine
Movie title found, ID: 976653, Title: Love in the Time of Fentanyl
Movie title found, ID: 1061671, Title: Pamela, A Love Story
Movie title found, ID: 622082, Title: In from the Side
Movie title found, ID: 714011, Title: After Love
Movie title found, ID: 804251, Title: Alcarràs
Movie title found, ID: 12652, Title: Nelly and Monsieur Arnaud
Movie title found, ID: 814338, Title: Lady Chatterley's Lover
Movie title found, ID: 1035923, Title: The Sound of Christmas
Movie title found, ID: 837938, Title:

In [15]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.418,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [16]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
print(tmdb_df)

                         title             original_title    budget  \
0       The Attachment Diaries                   El apego         0   
1         You Can Live Forever       You Can Live Forever         0   
2    A Tourist's Guide to Love  A Tourist's Guide to Love         0   
3      Other People's Children     Les Enfants des autres         0   
4               One True Loves             One True Loves         0   
..                         ...                        ...       ...   
149              The Last Face              The Last Face         0   
150              Lost in Paris            Paris pieds nus         0   
151             The Other Half             The Other Half         0   
152     The Ottoman Lieutenant     The Ottoman Lieutenant  40000000   
153                Art of Love            Romantik Hırsız         0   

    original_language                                           homepage  \
0                  es                                                  

### Merge and Clean the Data for Export

In [17]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on='title', how='inner')
display(merged_df)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.418,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",50.557,96,15055,2023-03-24,6.641,32,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,15.317,100,37820,2023-04-07,6.455,66,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,2.355,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,13.264,116,3395595,2023-03-31,6.900,80,"[Drama, Crime]","[English, Portuguese, Spanish]",[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",5.837,93,0,2017-09-15,5.800,40,"[Drama, Adventure]",[English],[]
123,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",14.778,130,0,2017-01-11,5.600,191,[Drama],"[English, Portuguese]",[United States of America]
124,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,10.038,83,0,2017-01-14,6.000,113,[Comedy],[French],"[Belgium, France]"
125,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21

In [18]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Create a list of characters to remove
characters_to_remove = ['[', ']', '\'']

# Loop through the list of columns to fix
for column in merged_df.columns:
    merged_df[column] = merged_df[column].astype(str)
    if column in columns_to_fix:
        # Convert the column to type 'str'
        merged_df[column] = merged_df[column].astype(str)

        # Loop through characters to remove
        for char in characters_to_remove: 
            merged_df[column] = merged_df[column].str.replace("[\[\]']", "", regex=True)

# Display the fixed DataFrame
display(merged_df.head(5))


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.418,102,0,2021-10-07,3.0,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",50.557,96,15055,2023-03-24,6.641,32,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,15.317,100,37820,2023-04-07,6.455,66,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,2.355,95,77145,2023-04-13,6.0,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,13.264,116,3395595,2023-03-31,6.9,80,"Drama, Crime","English, Portuguese, Spanish",United States of America


In [19]:
# Drop "byline.person" column
merged_df = merged_df.drop('byline.person', axis=1)

In [20]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.418,102,0,2021-10-07,3.0,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",50.557,96,15055,2023-03-24,6.641,32,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,15.317,100,37820,2023-04-07,6.455,66,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,2.355,95,77145,2023-04-13,6.0,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,13.264,116,3395595,2023-03-31,6.9,80,"Drama, Crime","English, Portuguese, Spanish",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",5.837,93,0,2017-09-15,5.8,40,"Drama, Adventure",English,
123,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",14.778,130,0,2017-01-11,5.6,191,Drama,"English, Portuguese",United States of America
124,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,10.038,83,0,2017-01-14,6.0,113,Comedy,French,"Belgium, France"
125,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The

In [21]:
# Export data to CSV without the index
merged_df.to_csv('movie_reviews.csv')